# Executar Simulações de Deposição

In [5]:
# importação dos objetos
from deposicao import DeposicaoAleatoria, DeposicaoAleatoriaRelaxacaoSuperficial, DeposicaoBalistica
from rng import RNG_PCG
import numpy as np
import pdb
import time
from multiprocessing import Pool

def simular(deposicao):
    deposicao.simular(verbose = False)
    return deposicao

'''
    Descomente a forma de deposição desejada (Aleatoria, Aleatoria com Relaxação
    Superficial ou Balística) e comente as demais.
'''
#forma_deposicao = DeposicaoAleatoria
forma_deposicao = DeposicaoAleatoriaRelaxacaoSuperficial
#forma_deposicao = DeposicaoBalistica

# numero de nucleos de processamento disponiveis, no nosso caso 4
instancias_paralelas = 4

# o numero de amostras totais, multiplo dos nucleos de processamento
# disponíveis
amostras = 32 * instancias_paralelas

# tempo minimo 40000
t_min = 1 * 10 ** 4

# conseguimos simular subestratos distintos
# recomendamos que simulem de 1 em 1 em cada
# simulação
comprimentos_subestratos = [200, 400]
#comprimentos_subestratos = [200, 400, 800, 1600]

if forma_deposicao == DeposicaoAleatoria:
    # na deposição aleatoria, a rugosidade nunca satura, por isso, não
    # faz sentido tempo_maximo > t_min
    z = 0
    coeficiente_tempo_maximo = 0
else:
    coeficiente_tempo_maximo = 1.1
    if forma_deposicao == DeposicaoAleatoriaRelaxacaoSuperficial:
        # na deposicao aleatoria com relaxacao superficial ocorre saturacao
        # da rugosidade quando t > L**2. Neste sentido, , tempo_maximo = 
        # max(10**4, 1.1*L**2)
        z = 2
    elif forma_deposicao == DeposicaoBalistica:
        # na deposicao balistica ocorre saturacao da rugosidade quando t > L**(3/2).
        # Neste sentido, , tempo_maximo = max(10**4, 1.1*L**(3/2))
        z = 3/2

# quantidade de snapshots (ou de instantanenos da simulação da deposição) para
# armazenarmos os dados do nosso interesse: instantaneos do crescimento da in-
# face e rugosidades
janelas = 20000   

t_max = lambda L, z = z, c = coeficiente_tempo_maximo, j = janelas:\
    np.int32(np.round(c * (L ** z)) - np.round(c * (L ** z)) % j)

tempo_maximo = {
    # setando tempo_maximo para cada tipo comprimento de subestrato definido em
    # comprimentos_subestratos
    L: max(t_min, t_max(L))
    for L in comprimentos_subestratos
}

dados_medios = {
    L:  {
        'interfaces': np.zeros(L),
        'tempo_maximo': tempo_maximo[L],
        'snapshots': janelas,
        'evolucao_subestratos': np.zeros((janelas ,L)),
        'rugosidades': np.zeros((janelas)),
        'amostras': amostras
    }
    for L in comprimentos_subestratos
}

# instanciando geradores de numeros aleatorios
rngs = [RNG_PCG() for i in range(instancias_paralelas)]

t_inicio = time.time()
for L in comprimentos_subestratos:
    tempo_maximo = dados_medios[L]['tempo_maximo']
    snapshots = dados_medios[L]['snapshots']
    
    # avancar o estado dos geradores de numeros aleatorios para cada instancia
    for i in range(1, instancias_paralelas):
        rngs[i].fast_forward(i * tempo_maximo)
    
    future_steps = 1
    for amostra in range(0,amostras,instancias_paralelas):
        # instancio instancias_paralelas de objetos de deposicao aleatoria
        instancias = []
        for i in range(instancias_paralelas):
            instancias.append(
                forma_deposicao(i, L, rngs[i], tempo_maximo = tempo_maximo, snapshots = snapshots )
            )
        
        begin = time.time()
        # simulacao de instancias_paralelas deposicoes
        print('L: {}, Simulacao das amostras {} a {}'.format(L, amostra + 1, amostra + instancias_paralelas))
        pool = Pool()
        deposicoes = pool.map(simular, instancias)
        pool.close()
        pool.join()
        print('Tempo Gasto: {:.2f}s'.format(time.time() - begin))
        
        # armazenar resultados
        for d in deposicoes:
            dados_medios[L]['evolucao_subestratos'] += d.altura_sitios / amostras
            dados_medios[L]['interfaces'] += d.altura_interface / amostras
            dados_medios[L]['rugosidades'] += d.rugosidade() / amostras
        
        # avancar geradores de numeros aleatorios
        for i in range(instancias_paralelas):
            rngs[i].fast_forward((i + future_steps * instancias_paralelas) * tempo_maximo)
        future_steps += 1
print('Tempo Total Gasto: {:.2f}s'.format(time.time() - t_inicio))

L: 400, Simulacao das amostras 1 a 4
Tempo Gasto: 9.22s
L: 400, Simulacao das amostras 5 a 8
Tempo Gasto: 9.18s
L: 400, Simulacao das amostras 9 a 12
Tempo Gasto: 13.08s
L: 400, Simulacao das amostras 13 a 16
Tempo Gasto: 12.33s
L: 400, Simulacao das amostras 17 a 20
Tempo Gasto: 12.68s
L: 400, Simulacao das amostras 21 a 24
Tempo Gasto: 13.05s
L: 400, Simulacao das amostras 25 a 28
Tempo Gasto: 11.55s
L: 400, Simulacao das amostras 29 a 32
Tempo Gasto: 11.64s
L: 400, Simulacao das amostras 33 a 36
Tempo Gasto: 16.17s
L: 400, Simulacao das amostras 37 a 40
Tempo Gasto: 15.17s
L: 400, Simulacao das amostras 41 a 44
Tempo Gasto: 15.05s
L: 400, Simulacao das amostras 45 a 48
Tempo Gasto: 13.47s
L: 400, Simulacao das amostras 49 a 52
Tempo Gasto: 13.01s
L: 400, Simulacao das amostras 53 a 56
Tempo Gasto: 13.50s
L: 400, Simulacao das amostras 57 a 60
Tempo Gasto: 13.27s
L: 400, Simulacao das amostras 61 a 64
Tempo Gasto: 13.12s
L: 400, Simulacao das amostras 65 a 68
Tempo Gasto: 13.60s
L: 4

# Salvar os Dados Para Uso Posterior

In [6]:
import pickle

diretorio = {
    DeposicaoAleatoria: 'DA',
    DeposicaoAleatoriaRelaxacaoSuperficial: 'DARS',
    DeposicaoBalistica: 'DB'
}

for L in comprimentos_subestratos:
    f = open('simulacoes/'+ diretorio[forma_deposicao] +'/L_{}'.format(L), 'wb')
    pickle.dump(dados_medios[L], f)
    f.close()